In [5]:
import os
import json
import pandas as pd
import traceback
from 


ImportError: attempted relative import with no known parent package

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
llm=ChatOpenAI(openai_api_key=keys.openaikey, model_name="gpt-3.5-turbo",temperature=0.3)

NameError: name 'keys' is not defined